# Usage Notes
This notebook should be launched from a session of jupyter notebook that was launched from a DB2 command window
To do this, run an administrator DB2 command window as an administrator and type 'jupyter notebook'

If you don't know what to do right now, try:
1. Update the database connectivity file with your database connectivity information
1. Click on each code sell in the "Environment set up" section and select "Run Cells" from the Cell menu on the toolbar immediately below the name of this notebook.
1.  On the Kernel menu, select "Restart & Run All"

If you have set up your enviornment before or used this Notebook before, then select "Cell" on the toolbar just under the name of this notebook, and select "run all". If database credentials are properly defined, this will generate the snapshot data using values since the last database start.

To emulate monitor reset, click the link that says "Collect baselines (reset monitor switches all)" below

To generate values based on the reset values, click on the text that says "Generate snapshot based on reset values", and then select "Run all Below" on the Cell menu to generate snapshot data based on a previously collected baseline.

# Environment set up:

Download the Db2 Extensions from  https://github.com/DB2-Samples/db2jupyter and place db2.ipyndb in the same folder as this jupyter notebook.

In [ ]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

## Import the modules and load the SQL magic

In [ ]:
## This cell must be executed any time the Kernel is started or restarted
%run db2.ipynb
from IPython.display import HTML

In [ ]:
## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [ ]:
baselines=0
user='db2admin'
host='localhost'
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename
password = LocalDB2password
port=50000
db='SAMPLE'
%sql CONNECT TO $db USER $user USING ? HOST $host PORT 50000

# Collect Baselines

In [ ]:
## Collect Baselines
HTML('''<script>
collect_baseline=false; 
function collect_baselines() {
 var kernel = IPython.notebook.kernel;
 collect_baseline=true;
 if (collect_baseline){
 kernel.execute('baseline_ts = %sql select current timestamp from sysibm.sysdummy1');
 kernel.execute('mgd_baseline = %sql select * from table(mon_get_database(-2)) as mgd');
 kernel.execute('mgtl_baseline = %sql select * from table(mon_get_transaction_log(-2)) as mgtl');
 kernel.execute('mgbp_baseline = %sql select * from table(mon_get_bufferpool(NULL,-2)) mgbp');
 kernel.execute('baselines=1');
 window.alert('Baselines Collected')
 }
} 

</script>
The baselines are not collected by default.
<a href="javascript:collect_baselines()">Collect baselines (reset monitor switches all)</a>.
''')

Generate snapshot based on reset values: Click on this text and then select "Run all Below" on the Cell menu to generate snapshot data based on a previously collected baseline.

In [ ]:
## To reset the monitor values, run this cell
## Baseline
#baseline_ts = %sql select current timestamp from sysibm.sysdummy1
#mgd_baseline = %sql select * from table(mon_get_database(-2)) as mgd
#mgtl_baseline = %sql select * from table(mon_get_transaction_log(-2)) as mgtl
#mgbp_baseline = %sql select * from table(mon_get_bufferpool(NULL,-2)) mgbp
#baselines=1

try:
    baseline_ts
except NameError:
    print("No baseline values defined. Data is since last database activation")
else:
    print(baseline_ts)

# Db2 Snapshot
Ember's snapshot for Db2 (LUW)

## Instance Uptime

In [ ]:
!db2pd -

## Database active since

In [ ]:
db_conn_time=%sql select db_conn_time from table(mon_get_database(-2)) as mgd
db_conn_time

## Five most recent activation times

In [ ]:
## Not working, come back to it
%sql select * \
from table(pd_get_log_msgs(current timestamp - 7 days, -2))\
where DBNAME= '{db}' 


## Last Succesful Backup

In [ ]:
%sql with ts as (select EID from sysibmadm.db_history where start_time = (select max(timestamp(start_time)) as ts  \
		from sysibmadm.db_history \
		where operation='B' and SQLCODE is NULL)\
	) \
select case \
		when operationtype = 'D' then 'DELTA_OFFLINE'\
		when operationtype = 'E' then 'DELTA_ONLINE' \
		when operationtype = 'F' then 'OFFLINE' \
		when operationtype = 'I' then 'INCR_OFFLINE' \
		when operationtype = 'N' then 'ONLINE' \
		when operationtype = 'O' then 'INCR_ONLINE' \
		else operationtype \
	end as type \
        , to_char(timestamp(start_time), 'YYYY-MM-DD HH24:MI:SS')  as backup_time \
FROM sysibmadm.db_history dh \
	join ts on dh.eid = ts.eid \
ORDER BY start_time desc \
fetch first 1 row only

## HADR details

In [ ]:
#untested - test on HADR database
%sql select hadr_role \
    , hadr_state \
    , hadr_connect_status \
    , hadr_log_gap \
FROM table(mon_get_hadr(-2))

## Tablespaces

In [ ]:
%sql select count(*) as NUM_TBSPS_NOT_AUTORESIZE \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_AUTO_RESIZE_ENABLED=0 \
    AND TBSP_TYPE='DMS'

In [ ]:
%sql select tbsp_name \
    , decimal(float(tbsp_used_pages)/float(tbsp_usable_pages),5,2) as pct_full \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_TYPE='DMS' \
    AND (TBSP_AUTO_RESIZE_ENABLED=0 OR TBSP_LAST_RESIZE_FAILED=1)

## Index Read Efficiency

In [ ]:
if baselines == 1:
    read_eff=%sql select decimal((float(rows_read)-{mgd_baseline.loc[0,"ROWS_READ"]})/(float(rows_returned)-{mgd_baseline.loc[0,"ROWS_READ"]}),16,2) read_eff \
    from table(mon_get_database(-2)) as mgd 
else:
    read_eff=%sql select decimal(float(rows_read)/float(rows_returned),16,2) read_eff \
    from table(mon_get_database(-2)) as mgd
read_eff

In [ ]:
#%sql select rows_modified \
#from table(mon_get_database(-2)) as mgd

In [ ]:
#%sql select rows_modified - {mgd_baseline[0].rows_modified} \
#from table(mon_get_database(-2)) as mgd

## Buffer Pool Hit Ratios

### Overall

In [ ]:
if baselines == 1:
    bp_details = %sql select \
        (pool_data_l_reads-{mgd_baseline.loc[0,"POOL_DATA_L_READS"]} + pool_xda_l_reads-{mgd_baseline.loc[0,"POOL_XDA_L_READS"]} + pool_index_l_reads-{mgd_baseline.loc[0,"POOL_INDEX_L_READS"]}) as total_l_reads \
        , case when pool_data_l_reads-{mgd_baseline.loc[0,"POOL_DATA_L_READS"]} > 0 or pool_temp_data_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_DATA_L_READS"]} > 0 \
            then decimal(((float(pool_data_lbp_pages_found-{mgd_baseline.loc[0,"POOL_DATA_LBP_PAGES_FOUND"]}) - float(pool_async_data_lbp_pages_found-{mgd_baseline.loc[0,"POOL_ASYNC_DATA_LBP_PAGES_FOUND"]})) / (float(pool_data_l_reads-{mgd_baseline.loc[0,"POOL_DATA_L_READS"]}) + float(pool_temp_data_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_DATA_L_READS"]}))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads-{mgd_baseline.loc[0,"POOL_INDEX_L_READS"]} > 0 or pool_temp_index_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_INDEX_L_READS"]} > 0 \
            then decimal(((float(pool_index_lbp_pages_found-{mgd_baseline.loc[0,"POOL_INDEX_LBP_PAGES_FOUND"]}) - float(pool_async_index_lbp_pages_found-{mgd_baseline.loc[0,"POOL_ASYNC_INDEX_LBP_PAGES_FOUND"]})) / (float(pool_index_l_reads-{mgd_baseline.loc[0,"POOL_INDEX_L_READS"]}) + float(pool_temp_index_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_INDEX_L_READS"]}))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads-{mgd_baseline.loc[0,"POOL_XDA_L_READS"]} > 0 or pool_temp_xda_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_XDA_L_READS"]} > 0 \
             then decimal(((float(pool_xda_lbp_pages_found-{mgd_baseline.loc[0,"POOL_XDA_LBP_PAGES_FOUND"]}) - float(pool_async_xda_lbp_pages_found-{mgd_baseline.loc[0,"POOL_ASYNC_XDA_LBP_PAGES_FOUND"]})) / (float(pool_xda_l_reads-{mgd_baseline.loc[0,"POOL_XDA_L_READS"]}) + float(pool_temp_xda_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_XDA_L_READS"]}))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads-{mgd_baseline.loc[0,"POOL_COL_L_READS"]} > 0 or pool_temp_col_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_COL_L_READS"]} > 0 \
             then decimal(((float(pool_col_lbp_pages_found-{mgd_baseline.loc[0,"POOL_COL_LBP_PAGES_FOUND"]}) - float(pool_async_col_lbp_pages_found-{mgd_baseline.loc[0,"POOL_ASYNC_COL_LBP_PAGES_FOUND"]})) / (float(pool_col_l_reads-{mgd_baseline.loc[0,"POOL_COL_L_READS"]}) + float(pool_temp_col_l_reads-{mgd_baseline.loc[0,"POOL_TEMP_COL_L_READS"]}))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_database(-2)) mgd 
    
else:
    bp_details = %sql select \
        (pool_data_l_reads + pool_xda_l_reads + pool_index_l_reads) as total_l_reads \
        , case when pool_data_l_reads > 0 or pool_temp_data_l_reads > 0 \
            then decimal(((float(pool_data_lbp_pages_found) - float(pool_async_data_lbp_pages_found)) / (float(pool_data_l_reads) + float(pool_temp_data_l_reads))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads > 0 or pool_temp_index_l_reads > 0 \
            then decimal(((float(pool_index_lbp_pages_found) - float(pool_async_index_lbp_pages_found)) / (float(pool_index_l_reads) + float(pool_temp_index_l_reads))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads > 0 or pool_temp_xda_l_reads > 0 \
             then decimal(((float(pool_xda_lbp_pages_found) - float(pool_async_xda_lbp_pages_found)) / (float(pool_xda_l_reads) + float(pool_temp_xda_l_reads))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads > 0 or pool_temp_col_l_reads > 0 \
             then decimal(((float(pool_col_lbp_pages_found) - float(pool_async_col_lbp_pages_found)) / (float(pool_col_l_reads) + float(pool_temp_col_l_reads))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_database(-2)) mgd
bp_details

### By Bufferpool

In [ ]:
df=None
df = pandas.DataFrame()
if baselines == 1:
    for ind, row in mgbp_baseline.iterrows():
        this_bp=row['BP_NAME']
        if this_bp.startswith("IBMSYS"):
            #skip this bp, as it is one of the small default ones
            meh=1
        else:
            df = %sql select bp_name \
                , (pool_data_l_reads-{row['POOL_DATA_L_READS']} + pool_xda_l_reads-{row['POOL_XDA_L_READS']} + pool_index_l_reads-{row['POOL_INDEX_L_READS']}) as total_l_reads \
                , case when pool_data_l_reads-{row['POOL_DATA_L_READS']} > 0 or pool_temp_data_l_reads-{row['POOL_TEMP_DATA_L_READS']} > 0 \
                    then decimal(((float(pool_data_lbp_pages_found-{row['POOL_DATA_LBP_PAGES_FOUND']}) - float(pool_async_data_lbp_pages_found-{row['POOL_ASYNC_DATA_LBP_PAGES_FOUND']})) / (float(pool_data_l_reads-{row['POOL_DATA_L_READS']}) + float(pool_temp_data_l_reads-{row['POOL_TEMP_DATA_L_READS']}))) * 100,5,2) \
                    else -1 end as data_hit_ratio_percent \
                , case when pool_index_l_reads-{row['POOL_INDEX_L_READS']} > 0 or pool_temp_index_l_reads-{row['POOL_TEMP_INDEX_L_READS']} > 0 \
                    then decimal(((float(pool_index_lbp_pages_found-{row['POOL_INDEX_LBP_PAGES_FOUND']}) - float(pool_async_index_lbp_pages_found-{row['POOL_ASYNC_INDEX_LBP_PAGES_FOUND']})) / (float(pool_index_l_reads-{row['POOL_INDEX_L_READS']}) + float(pool_temp_index_l_reads-{row['POOL_TEMP_INDEX_L_READS']}))) * 100,5,2) \
                    else -1 end as index_hit_ratio_percent \
                , case when pool_xda_l_reads-{row['POOL_XDA_L_READS']} > 0 or pool_temp_xda_l_reads-{row['POOL_TEMP_XDA_L_READS']} > 0 \
                     then decimal(((float(pool_xda_lbp_pages_found-{row['POOL_XDA_LBP_PAGES_FOUND']}) - float(pool_async_xda_lbp_pages_found-{row['POOL_ASYNC_XDA_LBP_PAGES_FOUND']})) / (float(pool_xda_l_reads-{row['POOL_XDA_L_READS']}) + float(pool_temp_xda_l_reads-{row['POOL_TEMP_XDA_L_READS']}))) * 100,5,2) \
                     else -1 end as xda_hit_ratio_percent \
                , case when pool_col_l_reads-{row['POOL_COL_L_READS']} > 0 or pool_temp_col_l_reads-{row['POOL_TEMP_COL_L_READS']} > 0 \
                     then decimal(((float(pool_col_lbp_pages_found-{row['POOL_COL_LBP_PAGES_FOUND']}) - float(pool_async_col_lbp_pages_found-{row['POOL_ASYNC_COL_LBP_PAGES_FOUND']})) / (float(pool_col_l_reads-{row['POOL_COL_L_READS']}) + float(pool_temp_col_l_reads-{row['POOL_TEMP_COL_L_READS']}))) * 100,5,2) \
                     else -1 end as col_hit_ratio_percent \
            from table(mon_get_bufferpool(NULL,-2)) mgbp \
            WHERE mgbp.bp_name not like 'IBMSYS%' \
                and mgbp.bp_name='{this_bp}'
        df = df.append(bp_details)
        bp_details=None
else:
    df = %sql select mgbp.bp_name \
        , (pool_data_l_reads + pool_xda_l_reads + pool_index_l_reads) as total_l_reads \
        , case when pool_data_l_reads > 0 or pool_temp_data_l_reads > 0 \
            then decimal(((float(pool_data_lbp_pages_found) - float(pool_async_data_lbp_pages_found)) / (float(pool_data_l_reads) + float(pool_temp_data_l_reads))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads > 0 or pool_temp_index_l_reads > 0 \
            then decimal(((float(pool_index_lbp_pages_found) - float(pool_async_index_lbp_pages_found)) / (float(pool_index_l_reads) + float(pool_temp_index_l_reads))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads > 0 or pool_temp_xda_l_reads > 0 \
             then decimal(((float(pool_xda_lbp_pages_found) - float(pool_async_xda_lbp_pages_found)) / (float(pool_xda_l_reads) + float(pool_temp_xda_l_reads))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads > 0 or pool_temp_col_l_reads > 0 \
             then decimal(((float(pool_col_lbp_pages_found) - float(pool_async_col_lbp_pages_found)) / (float(pool_col_l_reads) + float(pool_temp_col_l_reads))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_bufferpool(NULL,-2)) mgbp \
    WHERE mgbp.bp_name not like 'IBMSYS%'
    #return(bp_details)
    df = df.append(bp_details)
    bp_details=None
#bp_details
df.columns = ['BP Name', 'Logical Reads', 'Data Hit Ratio', 'Index Hit Ratio', 'XML Hit Ratio', 'Column Hit Ratio']
df

## Package Cache

In [ ]:
if baselines == 1:
    pkg_cache=%sql select \
        case when pkg_cache_lookups-{mgd_baseline.loc[0,"PKG_CACHE_LOOKUPS"]} > 0 \
            then decimal((1-(float(pkg_cache_inserts-{mgd_baseline.loc[0,"PKG_CACHE_INSERTS"]})/float(pkg_cache_lookups-{mgd_baseline.loc[0,"PKG_CACHE_LOOKUPS"]})))*100,5,2)  \
            else -1 end as pkg_cache_hitratio \
        , pkg_cache_num_overflows-{mgd_baseline.loc[0,"PKG_CACHE_NUM_OVERFLOWS"]} as pkg_cache_num_overflows \
        from table(mon_get_database(-2)) as mgd
else:
    pkg_cache=%sql select decimal((1-(float(pkg_cache_inserts)/float(pkg_cache_lookups)))*100,5,2) as pkg_cache_hitratio \
        , pkg_cache_num_overflows \
    from table(mon_get_database(-2)) as mgd
pkg_cache


## Catalog Cache

In [ ]:
if baselines == 1:
    cat_cache=%sql select \
        case when cat_cache_lookups-{mgd_baseline.loc[0,"CAT_CACHE_LOOKUPS"]} > 0 \
            then decimal((1-(float(cat_cache_inserts-{mgd_baseline.loc[0,"CAT_CACHE_INSERTS"]})/float(cat_cache_lookups-{mgd_baseline.loc[0,"PKG_CACHE_LOOKUPS"]})))*100,5,2)  \
            else -1 end as cat_cache_hitratio \
        , cat_cache_overflows-{mgd_baseline.loc[0,"CAT_CACHE_OVERFLOWS"]} as cat_cache_overflows \
        from table(mon_get_database(-2)) as mgd
else:
    cat_cache=%sql select decimal((1-(float(cat_cache_inserts)/float(cat_cache_lookups)))*100,5,2) as cat_cache_hitratio \
        , cat_cache_overflows \
    from table(mon_get_database(-2)) as mgd
cat_cache

## Transaction Logs

In [ ]:
if baselines == 1:
    tl_data= %sql select log_reads-{mgtl_baseline.loc[0,"LOG_READS"]} as log_reads \
    , log_writes-{mgtl_baseline.loc[0,"LOG_WRITES"]} as log_writes \
from table(mon_get_transaction_log(-2)) as mgtl 
else:
    tl_data= %sql select log_reads \
        , log_writes \
    from table(mon_get_transaction_log(-2)) as mgtl
tl_data

In [ ]:
%sql select decimal(100*(float(total_log_used)/float(total_log_available)),5,2) as tot_log_use_pct \
        , sec_logs_allocated \
        , sec_log_used_top \
    from table(mon_get_transaction_log(-2)) as mgtl

In [ ]:
%sql with t1 as (select substr(start_time, 1, 10) as ts \
  ,count(*) as archives \
from sysibmadm.db_history \
where operation = 'X' \
   and start_time > current timestamp - 7 days \
group by substr(start_time, 1, 10) ) \
select avg(archives) as avg_arch_per_hour \
    , max(archives) as max_arch_in_hour \
    , min(archives) as min_arch_in_hour \
from t1 \

## Locking

In [ ]:
if baselines == 1:
    locking= %sql select deadlocks-{mgd_baseline.loc[0,"DEADLOCKS"]} as deadlocks \
        , lock_timeouts-{mgd_baseline.loc[0,"LOCK_TIMEOUTS"]} as lock_timeouts \
        , lock_escals-{mgd_baseline.loc[0,"LOCK_ESCALS"]} as lock_escals\
    from table(mon_get_database(-2)) as mgd
else:
     locking= %sql select deadlocks \
        , lock_timeouts \
        , lock_escals \
    from table(mon_get_database(-2)) as mgd
locking

## Sorts

In [ ]:
if baselines == 1:
    sorts= %sql select \
        case when total_sorts-{mgd_baseline.loc[0,"TOTAL_SORTS"]} > 0 \
            then decimal(float(sort_overflows-{mgd_baseline.loc[0,"SORT_OVERFLOWS"]})/float(total_sorts-{mgd_baseline.loc[0,"TOTAL_SORTS"]}),5,2) \
            else 0 end as sort_overflow_pct \
    from table(mon_get_database(-2)) as mgd
else:
    sorts= %sql select \
        case when total_sorts >0 \
            then decimal(float(sort_overflows)/float(total_sorts),5,2) \
            else 0 end as sort_overflow_pct \
    from table(mon_get_database(-2)) as mgd
sorts